In [1]:
import wandb
import pandas as pd
from loguru import logger
from tqdm import tqdm
from itertools import product
import numpy as np
from pathlib import Path
import functools
import json
from typing import Literal

from histaug.utils import RunningStats, cached_df

api = wandb.Api()

INDEX_COLS = ["target", "train_dataset", "test_dataset", "model", "feature_extractor", "augmentations", "seed"]

RENAME_MODELS = {
    "AttentionMIL": "AttMIL",
    "MeanAveragePooling": "Mean pool",
    "Transformer": "Transformer",
}
RENAME_FEATURE_EXTRACTORS = {
    "bt": "Lunit-BT",
    "swav": "Lunit-SwAV",
    "dino_p16": "Lunit-DINO",
    "ctranspath": "CTransPath",
    "owkin": "Phikon",
    "resnet50": "ResNet-50",
    "retccl": "RetCCL",
    "swin": "Swin",
    "vit": "ViT",
}

RESULTS_DIR = Path("/app/results")

# Collect results from `wandb`

In [2]:
def filter_runs(runs, filters: dict):
    return [run for run in runs if all(getattr(run, key, None) == value for key, value in filters.items())]


def summarize_run(run):
    # return dict(
    #     target=(column := run.config["dataset"]["targets"][0]["column"]),
    #     train_dataset=run.config["dataset"]["name"],
    #     test_dataset=run.config["test"]["dataset"]["name"],
    #     model=run.config["model"]["_target_"].split(".")[-1],
    #     feature_extractor=run.config["settings"]["feature_extractor"],
    #     augmentations=run.config["dataset"]["augmentations"]["name"],
    #     seed=run.config["seed"],
    #     train_auroc=run.summary[f"train/{column}/auroc"]["best"],
    #     val_auroc=run.summary[f"val/{column}/auroc"]["best"],
    #     test_auroc=run.summary[f"test/{column}/auroc"]["best"],
    # )

    history = run.history().groupby("epoch").first()
    best = history[~history.index.isna()].sort_values("val/loss", ascending=True).iloc[0]
    column = run.config["dataset"]["targets"][0]["column"]
    if f"test/{column}/auroc" in run.summary:
        test_auroc = run.summary[f"test/{column}/auroc"]["best"]
    else:
        test_auroc = history[f"test/{column}/auroc"].max()
    return dict(
        target=column,
        train_dataset=run.config["dataset"]["name"],
        test_dataset=run.config["test"]["dataset"]["name"],
        model=run.config["model"]["_target_"].split(".")[-1],
        feature_extractor=run.config["settings"]["feature_extractor"],
        augmentations=run.config["dataset"]["augmentations"]["name"],
        seed=run.config["seed"],
        train_auroc=best[f"train/{column}/auroc"],
        val_auroc=best[f"val/{column}/auroc"],
        test_auroc=test_auroc,
    )


@cached_df(lambda: "aurocs")
def load_aurocs():
    logger.info("Loading runs")
    runs = list(api.runs("histaug", order="+created_at", per_page=1000))
    runs = filter_runs(runs, {"state": "finished"})
    runs = [summarize_run(run) for run in tqdm(runs, desc="Loading run data")]
    runs = [run for run in runs if run is not None]
    df = pd.DataFrame(runs)
    df = df.set_index(INDEX_COLS).sort_index().drop_duplicates()
    return df


df = df_all = load_aurocs().drop_duplicates()
df = (
    df.reset_index()
    .groupby(["target", "train_dataset", "test_dataset", "model", "feature_extractor", "augmentations"])
    .filter(lambda x: sorted(x.seed.values) == list(range(5)))
    .set_index(df.index.names)
    .sort_index()
)
print("Removed runs:", len(df_all) - len(df))
print(
    df_all.index.difference(df.index)
    .to_frame(index=False)
    .groupby([x for x in df.index.names if x != "seed"])
    .seed.count()
)

2023-11-07 12:09:18.281 | INFO     | __main__:load_aurocs:42 - Loading runs
Loading run data:  58%|█████▊    | 2734/4725 [22:57<15:55,  2.08it/s]

In [3]:
df = df.query("target in ['BRAF', 'CDH1', 'KRAS', 'MSI', 'PIK3CA', 'SMAD4', 'TP53', 'subtype']")

# Compare original vs Macenko (vs Macenko_slidewise)

In [4]:
macenko = df.query("augmentations == 'Macenko_patchwise'")["test_auroc"].droplevel("augmentations")
# macenko = df.query("augmentations == 'Macenko_slidewise'")["test_auroc"].droplevel("augmentations")
orig = df.query("augmentations == 'none'")["test_auroc"].droplevel("augmentations")
# Mean diff across seeds
d = (
    (macenko - orig)
    .rename("test_auroc_diff")
    .reset_index()
    .drop(columns="seed")
    .groupby(["target", "train_dataset", "test_dataset", "model", "feature_extractor"])
    .agg(["mean", "std"])
)
o = (
    orig.rename("test_auroc_orig")
    .reset_index()
    .drop(columns="seed")
    .groupby(["target", "train_dataset", "test_dataset", "model", "feature_extractor"])
    .agg(["mean", "std"])
)
d = pd.concat([d, o], axis=1)
d.query("model == 'Transformer'")

test_auroc_diff  \
                                                                                      mean   
target  train_dataset     test_dataset       model       feature_extractor                   
BRAF    tcga_crc_BRAF     cptac_crc_BRAF     Transformer bt                      -0.012434   
                                                         ctranspath               0.013032   
                                                         dino_p16                -0.060306   
                                                         owkin                    0.032447   
                                                         resnet50                 0.036968   
...                                                                                    ...   
subtype tcga_brca_subtype cptac_brca_subtype Transformer resnet50                -0.034491   
                                                         retccl                   0.031092   
                                                         swav                     0.013397   
                                                         swin                    -0.029599   
                                                         vit                     -0.040996   

                                                                                      \
                                                                                 std   
target  train_dataset     test_dataset       model       feature_extractor             
BRAF    tcga_crc_BRAF     cptac_crc_BRAF     Transformer bt                 0.032997   
                                                         ctranspath         0.034159   
                                                         dino_p16           0.112090   
                                                         owkin              0.099511   
                                                         resnet50           0.062584   
...                                                                              ...   
subtype tcga_brca_subtype cptac_brca_subtype Transformer resnet50           0.038394   
                                                         retccl             0.039379   
                                                         swav               0.072871   
                                                         swin               0.030260   
                                                         vit                0.030121   

                                                                           test_auroc_orig  \
                                                                                      mean   
target  train_dataset     test_dataset       model       feature_extractor                   
BRAF    tcga_crc_BRAF     cptac_crc_BRAF     Transformer bt                       0.629920   
                                                         ctranspath               0.678457   
                                                         dino_p16                 0.737766   
                                                         owkin                    0.653457   
                                                         resnet50                 0.569282   
...                                                                                    ...   
subtype tcga_brca_subtype cptac_brca_subtype Transformer resnet50                 0.690357   
                                                         retccl                   0.727910   
                                                         swav                     0.737178   
                                                         swin                     0.736527   
                                                         vit                      0.743008   

                                                                                      
                                                                                 std  
target  train_dataset     test_dat

# What is the best feature extractor?

In [5]:
augmentation = "none"
augmentation = "Macenko_patchwise"
# augmentation = "Macenko_slidewise"

test_aurocs = df.query(f"augmentations == '{augmentation}'").droplevel("augmentations")["test_auroc"]
# test_aurocs = df.query(f"augmentations == '{augmentation}' and target == 'MSI'").droplevel("augmentations")["test_auroc"]
test_aurocs

target   train_dataset      test_dataset        model         feature_extractor  seed
BRAF     tcga_crc_BRAF      cptac_crc_BRAF      AttentionMIL  bt                 0       0.423537
                                                                                 1       0.655585
                                                                                 2       0.455452
                                                                                 3       0.450798
                                                                                 4       0.405585
                                                                                           ...   
subtype  tcga_brca_subtype  cptac_brca_subtype  Transformer   vit                0       0.731312
                                                                                 1       0.722702
                                                                                 2       0.657148
                                

In [6]:
def compute_norm_diff_auroc(sub_df):
    """Function to compute average offset from best for a given subset of data."""
    pivot_data = sub_df.pivot(index="seed", columns="feature_extractor", values="test_auroc")
    feature_extractors = pivot_data.columns.values
    seeds = pivot_data.index.values
    combinations = product(*pivot_data.values.T)
    n_combinations = int(len(seeds) ** len(feature_extractors))
    stats_by_feature_extractor = {fe: RunningStats() for fe in feature_extractors}

    for i, auroc_values in enumerate(tqdm(combinations, total=n_combinations)):
        # sorted_indices = np.argsort(auroc_values)[::-1]
        # ranks_array[sorted_indices] += np.arange(1, len(feature_extractors) + 1)
        diffs = np.array(auroc_values).max() - np.array(auroc_values)
        for fe, diff in zip(feature_extractors, diffs):
            stats_by_feature_extractor[fe].update(diff)

    return {fe: stats.compute() for fe, stats in stats_by_feature_extractor.items()}


@cached_df(lambda *args, **kwargs: f"norm_diff_auroc_{augmentation}")
def compute_results_table(test_aurocs: pd.Series):
    """Compute average offsets from best for each (target, model) pair."""
    d = test_aurocs.reset_index()

    results = {}
    unique_pairs = d[["target", "model"]].drop_duplicates().values

    for target, model in unique_pairs:
        sub_data = d[(d["target"] == target) & (d["model"] == model)]
        results[(target, model)] = compute_norm_diff_auroc(sub_data)
        print(
            f"{target:10s} {model:20s}:",
            ", ".join(
                f"{k}={mean:.2f}+-{std:.2f}"
                for (k, (mean, std)) in sorted(results[(target, model)].items(), key=lambda x: x[1], reverse=False)
            ),
        )

    r = pd.DataFrame(results).map(lambda x: x._asdict())
    r.index.name = "feature_extractor"
    r.columns.names = ["target", "model"]
    r = r.stack().stack().apply(pd.Series)
    r.columns.names = ["stats"]
    r = (
        r.pivot_table(index=["model", "feature_extractor"], columns="target")
        .reorder_levels([1, 0], axis=1)
        .sort_index(axis=1)
    )
    return r


r = compute_results_table(test_aurocs)
r

  0%|          | 0/1953125 [00:00<?, ?it/s]

100%|██████████| 1953125/1953125 [01:26<00:00, 22661.97it/s]


BRAF       AttentionMIL        : dino_p16=0.02+-0.04, ctranspath=0.06+-0.04, swav=0.07+-0.05, owkin=0.09+-0.05, swin=0.11+-0.07, vit=0.13+-0.06, retccl=0.14+-0.04, resnet50=0.17+-0.07, bt=0.28+-0.10


100%|██████████| 1953125/1953125 [01:23<00:00, 23320.14it/s]


BRAF       MeanAveragePooling  : dino_p16=0.00+-0.00, swav=0.04+-0.03, ctranspath=0.07+-0.04, owkin=0.09+-0.03, retccl=0.14+-0.05, swin=0.15+-0.04, vit=0.16+-0.06, bt=0.18+-0.03, resnet50=0.21+-0.04


...

100%|██████████| 1953125/1953125 [01:23<00:00, 23420.04it/s]


subtype    MeanAveragePooling  : ctranspath=0.00+-0.00, retccl=0.01+-0.00, vit=0.03+-0.00, dino_p16=0.05+-0.01, bt=0.06+-0.03, swav=0.06+-0.00, swin=0.06+-0.01, resnet50=0.08+-0.00, owkin=0.11+-0.01


100%|██████████| 1953125/1953125 [01:24<00:00, 23209.08it/s]
/tmp/ipykernel_2127000/244755881.py:42: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  r = r.stack().stack().apply(pd.Series)


subtype    Transformer         : ctranspath=0.01+-0.02, bt=0.03+-0.03, dino_p16=0.04+-0.03, retccl=0.06+-0.03, swav=0.06+-0.03, owkin=0.08+-0.03, swin=0.11+-0.04, vit=0.11+-0.03, resnet50=0.16+-0.05


target                                    BRAF                CDH1            \
stats                                     mean       std      mean       std   
model              feature_extractor                                           
AttentionMIL       bt                 0.278917  0.096484  0.040381  0.014883   
                   ctranspath         0.058704  0.036685  0.021078  0.029036   
                   dino_p16           0.022135  0.037561  0.009999  0.011961   
                   owkin              0.093810  0.045302  0.044607  0.013955   
                   resnet50           0.168677  0.070385  0.183557  0.036119   
                   retccl             0.141018  0.036274  0.034899  0.020714   
                   swav               0.072068  0.052840  0.026332  0.027243   
                   swin               0.107773  0.074119  0.196406  0.042599   
                   vit                0.126190  0.063989  0.115310  0.031582   
MeanAveragePooling bt                 0.181955  0.025563  0.021193  0.008257   
                   ctranspath         0.070718  0.038852  0.005202  0.006431   
                   dino_p16           0.000771  0.002911  0.016510  0.011786   
                   owkin              0.089867  0.033434  0.009086  0.009219   
                   resnet50           0.212939  0.044663  0.109714  0.044172   
                   retccl             0.136476  0.048729  0.016853  0.007492   
                   swav               0.042859  0.032428  0.019252  0.011039   
                   swin               0.146782  0.039910  0.124106  0.020555   
                   vit                0.159215  0.056099  0.087784  0.007742   
Transformer        bt                 0.148513  0.037883  0.040734  0.026102   
                   ctranspath         0.074510  0.066965  0.052042  0.036446   
                   dino_p16           0.088539  0.074985  0.048444  0.037183   
                   owkin              0.080095  0.042685  0.085166  0.031200   
                   resnet50           0.159749  0.052773  0.177170  0.099972   
                   retccl             0.145986  0.076523  0.057639  0.041600   
                   swav               0.073712  0.104211  0.013093  0.014527   
                   swin               0.217927  0.069700  0.187393  0.050929   
                   vit                0.212542  0.043601  0.127542  0.026785   

target                                    KRAS                 MSI            \
stats                                     mean       std      mean       std   
model              feature_extractor                                           
AttentionMIL       bt                 0.073060  0.056746  0.332843  0.073579   
                   ctranspath         0.061292  0.030044  0.071732  0.056848   
                   dino_p16           0.065046  0.039180  0.010466  0.016273   
                   owkin              0.050181  0.047491  0.028625  0.030363   
                   resnet50           0.121765  0.031194  0.180785  0.028316   
                   retccl             0.012231  0.022048  0.070188  0.034005   
                   swav               0.118124  0.076239  0.123481  0.039511   
                   swin               0.105474  0.036597  0.137472  0.026095   
                   vit                0.091134  0.039121  0.142719  0.038258   
MeanAveragePooling bt                 0.019133  0.027382  0.108877  0.037839   
                   ctranspath         0.088690  0.051140  0.028579  0.029546   
                   dino_p16           0.068608  0.035591  0.041799  0.038491   
                   owkin              0.047737  0.028328  0.016902  0.025836   
                   resnet50           0.022324  0.024852  0.147355  0.049016   
                   retccl             0.047362  0.045598  0.042005  0.027874   
                   swav               0.152016  0.016907  0.118137  0.032539   
                   swin               0.038803  0.032688  0.112787  0.034075   

In [18]:
# Computer overall mean and std (across targets)
n_targets = r.stack().query("stats == 'mean'").droplevel("stats").shape[1]
overall_mean = r.stack().query("stats == 'mean'").droplevel("stats").sum(axis="columns").divide(n_targets)
overall_std = r.stack().query("stats == 'std'").droplevel("stats").pow(2).sum(axis="columns").pow(0.5).divide(n_targets)
r["average", "mean"] = overall_mean
r["average", "std"] = overall_std
r

target                                    BRAF                CDH1            \
stats                                     mean       std      mean       std   
model              feature_extractor                                           
AttentionMIL       bt                 0.345998  0.133092  0.064523  0.035145   
                   ctranspath         0.060360  0.027964  0.006499  0.012455   
                   dino_p16           0.015280  0.024514  0.034255  0.034877   
                   owkin              0.069402  0.060869  0.089423  0.022736   
                   resnet50           0.226583  0.068262  0.087824  0.038983   
                   retccl             0.141078  0.030904  0.037567  0.016970   
                   swav               0.070399  0.065243  0.064295  0.030173   
                   swin               0.142407  0.065605  0.168407  0.025006   
                   vit                0.179442  0.074738  0.110668  0.019652   
MeanAveragePooling bt                 0.207247  0.090669  0.035422  0.006928   
                   ctranspath         0.060173  0.033205  0.038049  0.018305   
                   dino_p16           0.000000  0.000000  0.038734  0.022498   
                   owkin              0.086303  0.067060  0.040448  0.025984   
                   resnet50           0.235638  0.023230  0.000813  0.002900   
                   retccl             0.147074  0.014095  0.028112  0.008832   
                   swav               0.027593  0.011791  0.029026  0.015808   
                   swin               0.125200  0.027127  0.100014  0.037291   
                   vit                0.176729  0.036394  0.074029  0.010717   
Transformer        bt                 0.132337  0.052929  0.048161  0.025943   
                   ctranspath         0.083800  0.063167  0.013038  0.016322   
                   dino_p16           0.024492  0.036364  0.061981  0.026514   
                   owkin              0.108800  0.070751  0.075574  0.047996   
                   resnet50           0.192976  0.064142  0.101787  0.071861   
                   retccl             0.215782  0.109034  0.037367  0.041928   
                   swav               0.068375  0.048193  0.035653  0.053540   
                   swin               0.166978  0.053158  0.113438  0.025500   
                   vit                0.204013  0.080049  0.101273  0.021977   

target                                    KRAS                 MSI            \
stats                                     mean       std      mean       std   
model              feature_extractor                                           
AttentionMIL       bt                 0.032743  0.036754  0.277140  0.133136   
                   ctranspath         0.059225  0.032613  0.083982  0.030418   
                   dino_p16           0.061027  0.041159  0.001214  0.004766   
                   owkin              0.065269  0.042018  0.064743  0.043849   
                   resnet50           0.112228  0.033884  0.220247  0.042516   
                   retccl             0.028707  0.024362  0.081512  0.034558   
                   swav               0.128707  0.059317  0.102654  0.033832   
                   swin               0.140156  0.042411  0.176626  0.043431   
                   vit                0.029571  0.041542  0.146893  0.034201   
MeanAveragePooling bt                 0.020973  0.023927  0.080158  0.019227   
                   ctranspath         0.102692  0.050946  0.030621  0.021808   
                   dino_p16           0.080432  0.014821  0.001660  0.004789   
                   owkin              0.113240  0.030459  0.071362  0.034569   
                   resnet50           0.022024  0.036016  0.215446  0.029804   
                   retccl             0.023338  0.024927  0.099551  0.041940   
                   swav               0.123600  0.024708  0.053307  0.016429   
                   swin               0.016656  0.014586  0.173214  0.024998   

In [7]:
def results_to_latex(r: pd.DataFrame, goal: Literal["min", "max"] = "min") -> str:
    # Format for appearance
    r = r.unstack("model")
    means = r.stack("stats").query("stats == 'mean'").droplevel("stats")
    stds = r.stack("stats").query("stats == 'std'").droplevel("stats")
    formatted = means.map(lambda x: f"{x:.2f}") + " \\pm " + stds.map(lambda x: f"{x:.2f}")

    # Make best model bold
    best_mask = means == getattr(means, goal)(axis="index")
    formatted[best_mask] = "\\mathbf{" + formatted[best_mask] + "}"
    formatted = "$" + formatted + "$"
    formatted = formatted.stack("model")
    formatted = formatted.swaplevel("feature_extractor", "model").sort_index()

    # Check if we have average column
    targets = formatted.columns.get_level_values("target").unique()
    has_average = "average" in targets
    if has_average:
        targets = targets.drop("average")

    # Rename
    formatted = formatted.rename(RENAME_MODELS, level="model")
    formatted = formatted.rename(RENAME_FEATURE_EXTRACTORS, level="feature_extractor")
    formatted.index.names = ["Model", "Feature extractor"]
    formatted.columns.names = ["Target"]
    return formatted.to_latex(
        column_format="l" * len(formatted.index.levels) + "|" + "c" * len(targets) + ("|c" if has_average else ""),
        escape=False,
    )


print(results_to_latex(r))

\begin{tabular}{ll|cccc|c}
\toprule
 & Target & CDH1 & PIK3CA & TP53 & subtype & average \\
Model & Feature extractor &  &  &  &  &  \\
\midrule
\multirow[t]{9}{*}{AttMIL} & Lunit-BT & $0.04 \pm 0.01$ & $0.13 \pm 0.02$ & $\mathbf{0.02 \pm 0.02}$ & $0.11 \pm 0.04$ & $0.08 \pm 0.01$ \\
 & CTransPath & $0.02 \pm 0.03$ & $0.04 \pm 0.01$ & $0.03 \pm 0.02$ & $\mathbf{0.00 \pm 0.00}$ & $\mathbf{0.02 \pm 0.01}$ \\
 & Lunit-DINO & $\mathbf{0.01 \pm 0.01}$ & $0.02 \pm 0.02$ & $0.04 \pm 0.03$ & $0.04 \pm 0.03$ & $0.03 \pm 0.01$ \\
 & Phikon & $0.04 \pm 0.01$ & $0.05 \pm 0.02$ & $0.09 \pm 0.04$ & $0.11 \pm 0.03$ & $0.07 \pm 0.01$ \\
 & ResNet-50 & $0.18 \pm 0.04$ & $\mathbf{0.01 \pm 0.01}$ & $0.17 \pm 0.05$ & $0.17 \pm 0.04$ & $0.13 \pm 0.02$ \\
 & RetCCL & $0.03 \pm 0.02$ & $0.02 \pm 0.02$ & $0.03 \pm 0.04$ & $0.09 \pm 0.04$ & $0.04 \pm 0.02$ \\
 & Lunit-SwAV & $0.03 \pm 0.03$ & $0.05 \pm 0.03$ & $0.04 \pm 0.02$ & $0.05 \pm 0.03$ & $0.04 \pm 0.01$ \\
 & Swin & $0.20 \pm 0.04$ & $0.05 \pm 0.03$ & 

In [8]:
# Aggregate test AUROCS over seeds
t = test_aurocs.agg(["mean", "std"])
# means = t["mean"].unstack("target").unstack("model")
# stds = t["std"].unstack("target").unstack("model")

# formatted = means.map(lambda x: f"{x:.2f}") + " \\pm " + stds.map(lambda x: f"{x:.2f}")
# # Make best model bold
# best_mask = means == means.max(axis="index")
# formatted[best_mask] = "\\mathbf{" + formatted[best_mask] + "}"
# formatted = "$" + formatted + "$"
# formatted = formatted.stack("model")
# formatted = formatted.swaplevel("feature_extractor", "model").sort_index()
# formatted = formatted.rename(RENAME_MODELS, level="model")
# formatted = formatted.rename(RENAME_FEATURE_EXTRACTORS, level="feature_extractor")
# formatted.index.names = ["Model", "Feature extractor"]
# formatted.columns.names = ["Target"]
# print(formatted.to_latex(column_format="l" * len(formatted.index.levels) + "|" + "c" * n_targets + "|c", escape=False))

# print(results_to_latex(r))
t = (
    test_aurocs.droplevel(["train_dataset", "test_dataset", "seed"])
    .reset_index()
    .groupby(["target", "model", "feature_extractor"])
    .agg(["mean", "std"])
    .droplevel(0, axis="columns")
)
t.columns.names = ["stats"]
t = t.unstack("target").swaplevel("stats", "target", axis="columns").sort_index(axis="columns")
print(results_to_latex(t, goal="max"))

\begin{tabular}{ll|cccccc}
\toprule
 & Target & CDH1 & KRAS & MSI & PIK3CA & TP53 & subtype \\
Model & Feature extractor &  &  &  &  &  &  \\
\midrule
\multirow[t]{9}{*}{AttMIL} & Lunit-BT & $0.76 \pm 0.01$ & $nan \pm nan$ & $0.57 \pm 0.08$ & $0.51 \pm 0.02$ & $\mathbf{0.79 \pm 0.03}$ & $0.70 \pm 0.03$ \\
 & CTransPath & $0.78 \pm 0.04$ & $0.61 \pm 0.03$ & $0.83 \pm 0.06$ & $0.60 \pm 0.01$ & $0.78 \pm 0.02$ & $\mathbf{0.81 \pm 0.03}$ \\
 & Lunit-DINO & $\mathbf{0.79 \pm 0.01}$ & $nan \pm nan$ & $\mathbf{0.89 \pm 0.03}$ & $0.62 \pm 0.02$ & $0.77 \pm 0.03$ & $0.77 \pm 0.02$ \\
 & Phikon & $0.76 \pm 0.01$ & $0.64 \pm 0.03$ & $0.87 \pm 0.03$ & $0.59 \pm 0.02$ & $0.72 \pm 0.04$ & $0.70 \pm 0.02$ \\
 & ResNet-50 & $0.62 \pm 0.04$ & $0.54 \pm nan$ & $0.72 \pm 0.02$ & $\mathbf{0.63 \pm 0.01}$ & $0.64 \pm 0.06$ & $0.64 \pm 0.03$ \\
 & RetCCL & $0.77 \pm 0.02$ & $\mathbf{0.65 \pm nan}$ & $0.83 \pm 0.03$ & $0.62 \pm 0.02$ & $0.78 \pm 0.05$ & $0.73 \pm 0.03$ \\
 & Lunit-SwAV & $0.78 \pm 0.03$ & $n

In [9]:
augmentation

'Macenko_patchwise'